# Pyspark TensorFlow Inference

## Feature Columns
From: https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers

### Using TensorFlow
Note that cuFFT/cuDNN/cuBLAS registration errors are expected with `tf=2.17.0` and will not affect behavior, as noted in [this issue.](https://github.com/tensorflow/tensorflow/issues/62075)  
This notebook does not demonstrate inference with TensorRT, as [TF-TRT](https://docs.nvidia.com/deeplearning/tensorrt/release-notes/index.html#tensorrt-10) does not yet support `tf=2.17.0`. See the `pytorch` notebooks for TensorRT demos.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

2024-10-24 16:04:17.711230: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 16:04:17.719701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 16:04:17.728758: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 16:04:17.731459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 16:04:17.738797: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print(tf.__version__)

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2.17.0


In [7]:
import pathlib
import os
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'

data_dir = tf.keras.utils.get_file('petfinder_mini.zip', dataset_url, extract=True, cache_dir='.')
data_dir = pathlib.Path(data_dir)
try:
    # pet-finder-mini might be under a parent a directory petfinder_mini_extracted. Check if this is the case:
    dataset = os.path.join(os.path.dirname(data_dir), 'petfinder_mini_extracted/petfinder-mini/petfinder-mini.csv')
    dataframe = pd.read_csv(dataset)
except:
    dataset = os.path.join(os.path.dirname(data_dir), 'petfinder-mini/petfinder-mini.csv')
    dataframe = pd.read_csv(dataset)

In [11]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [6]:
# In the original dataset, `'AdoptionSpeed'` of `4` indicates
# a pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop unused features.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

In [7]:
train, val, test = np.split(dataframe.sample(frac=1), [int(0.8*len(dataframe)), int(0.9*len(dataframe))])

/home/rishic/anaconda3/envs/spark-dl-tf/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [8]:
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

9229 training examples
1154 validation examples
1154 test examples


In [9]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('target')
    df = {key: value.to_numpy()[:,tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [10]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

2024-10-03 17:38:53.526119: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46022 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6


In [11]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of targets:', label_batch )

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']
A batch of ages: tf.Tensor(
[[18]
 [ 5]
 [ 2]
 [ 5]
 [ 1]], shape=(5, 1), dtype=int64)
A batch of targets: tf.Tensor([1 0 1 1 1], shape=(5,), dtype=int64)


2024-10-03 17:38:53.588272: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

In [13]:
photo_count_col = train_features['PhotoAmt']
layer = get_normalization_layer('PhotoAmt', train_ds)
layer(photo_count_col)

2024-10-03 17:38:55.015073: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.19597158],
       [-0.19597158],
       [-0.8320273 ],
       [-0.19597158],
       [-0.8320273 ]], dtype=float32)>

In [14]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # Create a layer that turns strings into integer indices.
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    # Otherwise, create a layer that turns integer values into integer indices.
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)

    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)

    # Encode the integer indices.
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    return lambda feature: encoder(index(feature))

In [15]:
test_type_col = train_features['Type']
test_type_layer = get_category_encoding_layer(name='Type',
                                              dataset=train_ds,
                                              dtype='string')
test_type_layer(test_type_col)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)>

In [16]:
test_age_col = train_features['Age']
test_age_layer = get_category_encoding_layer(name='Age',
                                             dataset=train_ds,
                                             dtype='int64',
                                             max_tokens=5)
test_age_layer(test_age_col)

2024-10-03 17:38:56.454126: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)>

In [17]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [18]:
all_inputs = {}
encoded_features = []

# Numerical features.
for header in ['PhotoAmt', 'Fee']:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs[header] = numeric_col
    encoded_features.append(encoded_numeric_col)

In [19]:
age_col = tf.keras.Input(shape=(1,), name='Age', dtype='int64')

encoding_layer = get_category_encoding_layer(name='Age',
                                             dataset=train_ds,
                                             dtype='int64',
                                             max_tokens=5)
encoded_age_col = encoding_layer(age_col)
all_inputs['Age'] = age_col
encoded_features.append(encoded_age_col)

In [20]:
categorical_cols = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                    'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Breed1']

for header in categorical_cols:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(name=header,
                                                dataset=train_ds,
                                                dtype='string',
                                                max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs[header] = categorical_col
    encoded_features.append(encoded_categorical_col)

2024-10-03 17:38:56.758056: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-10-03 17:38:57.171981: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)

In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"],
              run_eagerly=True)

In [23]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4109 - loss: 0.7333 - val_accuracy: 0.6898 - val_loss: 0.5666
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6423 - loss: 0.5994 - val_accuracy: 0.7210 - val_loss: 0.5484
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6825 - loss: 0.5728 - val_accuracy: 0.7253 - val_loss: 0.5383
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6796 - loss: 0.5653 - val_accuracy: 0.7331 - val_loss: 0.5314
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6853 - loss: 0.5584 - val_accuracy: 0.7348 - val_loss: 0.5259
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7120 - loss: 0.5447 - val_accuracy: 0.7418 - val_loss: 0.5218
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7068 - loss: 0.5422 - val_accuracy: 0.7435 - val_loss: 0.5189
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7043 - loss: 0.5397 - val_accuracy: 0.7435 - val_loss: 0.

In [24]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7480 - loss: 0.5028 
Accuracy 0.753032922744751


## Save and Reload Model

In [25]:
import tensorflow as tf

In [26]:
model.save('my_pet_classifier.keras')

In [27]:
reloaded_model = tf.keras.models.load_model('my_pet_classifier.keras')

In [28]:
sample = {
    'Type': 'Cat',
    'Age': 3,
    'Breed1': 'Tabby',
    'Gender': 'Male',
    'Color1': 'Black',
    'Color2': 'White',
    'MaturitySize': 'Small',
    'FurLength': 'Short',
    'Vaccinated': 'No',
    'Sterilized': 'No',
    'Health': 'Healthy',
    'Fee': 100,
    'PhotoAmt': 2,
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(
    "This particular pet had a %.1f percent probability "
    "of getting adopted." % (100 * prob)
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
This particular pet had a 81.1 percent probability of getting adopted.


## PySpark

In [29]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [ ]:
import os
conda_env = os.environ.get("CONDA_PREFIX")

conf = SparkConf()
if 'spark' not in globals():
    # If Spark is not already started with Jupyter, attach to Spark Standalone
    import socket
    hostname = socket.gethostname()
    conf.setMaster(f"spark://{hostname}:7077") # assuming Master is on default port 7077
conf.set("spark.task.maxFailures", "1")
conf.set("spark.driver.memory", "8g")
conf.set("spark.executor.memory", "8g")
conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")
conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled", "false")
conf.set("spark.sql.pyspark.jvmStacktrace.enabled", "true")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.python.worker.reuse", "true")
# Create Spark Session
spark = SparkSession.builder.appName("spark-dl-examples").config(conf=conf).getOrCreate()
sc = spark.sparkContext

In [31]:
df = spark.createDataFrame(dataframe).repartition(8)

In [32]:
df.write.mode("overwrite").parquet("datasets/petfinder-mini")

In [33]:
df.show()

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized| Health|Fee|PhotoAmt|target|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
| Cat|  3|               Tabby|  Male| Black|   White|       Small|    Short|        No|        No|Healthy|100|       1|     1|
| Cat|  1|Domestic Medium Hair|  Male| Black|   Brown|      Medium|   Medium|  Not Sure|  Not Sure|Healthy|  0|       2|     1|
| Dog|  1|         Mixed Breed|  Male| Brown|   White|      Medium|   Medium|       Yes|        No|Healthy|  0|       7|     1|
| Dog|  4|         Mixed Breed|Female| Black|   Brown|      Medium|    Short|       Yes|        No|Healthy|150|       8|     1|
| Dog|  1|         Mixed Breed|  Male| Black|No Color|      Medium|    Short|        No|        No|Healt

## Inference using Spark DL API

In [34]:
import numpy as np
import os

from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct, col
from pyspark.sql.types import ArrayType, FloatType

In [35]:
df = spark.read.parquet("datasets/petfinder-mini").cache()
df.show(5)

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized| Health|Fee|PhotoAmt|target|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
| Cat|  1|Domestic Medium Hair|Female| White|No Color|       Small|   Medium|        No|        No|Healthy|  0|       2|     1|
| Dog|  2|         Mixed Breed|Female| Black|   Brown|      Medium|   Medium|        No|        No|Healthy|  0|       3|     1|
| Dog| 18|           Dalmatian|Female| Black|   White|      Medium|   Medium|       Yes|        No|Healthy|350|       5|     1|
| Dog|  3|         Mixed Breed|Female| Black|No Color|      Medium|    Short|        No|        No|Healthy|  0|       1|     0|
| Dog|  2|         Mixed Breed|  Male| Black|   Brown|      Medium|    Short|        No|        No|Healt

In [36]:
columns = df.columns
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']


In [37]:
# remove label column
columns.remove("target")
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']


In [38]:
# get absolute path to model
model_dir = "{}/my_pet_classifier.keras".format(os.getcwd())

In [39]:
def predict_batch_fn():
    import tensorflow as tf
    import pandas as pd
    
    # Enable GPU memory growth to avoid CUDA OOM
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    model = tf.keras.models.load_model(model_dir)

    def predict(t, a, b, g, c1, c2, m, f, v, s, h, fee, p):
        inputs = {
            "Type": t,
            "Age": a,
            "Breed1": b,
            "Gender": g,
            "Color1": c1,
            "Color2": c2,
            "MaturitySize": m,
            "FurLength": f,
            "Vaccinated": v,
            "Sterilized": s,
            "Health": h,
            "Fee": fee,
            "PhotoAmt": p
        }
        # return model.predict(inputs)
        return pd.Series(np.squeeze(model.predict(inputs)))

    return predict

In [40]:
# need to pass the list of columns into the model_udf
classify = predict_batch_udf(predict_batch_fn,
                             return_type=FloatType(),
                             batch_size=100)

In [41]:
%%time
preds = df.withColumn("preds", classify(struct(*columns)))
results = preds.collect()

24/10/03 17:39:09 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CPU times: user 24.1 ms, sys: 6.37 ms, total: 30.4 ms
Wall time: 4.58 s


In [42]:
%%time
preds = df.withColumn("preds", classify(*columns))
results = preds.collect()

CPU times: user 98.2 ms, sys: 8.34 ms, total: 107 ms
Wall time: 1.57 s


In [43]:
%%time
preds = df.withColumn("preds", classify(*[col(c) for c in columns]))
results = preds.collect()

CPU times: user 17.6 ms, sys: 2.19 ms, total: 19.8 ms
Wall time: 1.51 s


In [44]:
preds.show()

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+------------+---+--------+------+-----------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized|      Health|Fee|PhotoAmt|target|      preds|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+------------+---+--------+------+-----------+
| Cat|  1|Domestic Medium Hair|Female| White|No Color|       Small|   Medium|        No|        No|     Healthy|  0|       2|     1|  1.9543833|
| Dog|  2|         Mixed Breed|Female| Black|   Brown|      Medium|   Medium|        No|        No|     Healthy|  0|       3|     1|  1.7454995|
| Dog| 18|           Dalmatian|Female| Black|   White|      Medium|   Medium|       Yes|        No|     Healthy|350|       5|     1|  1.5183508|
| Dog|  3|         Mixed Breed|Female| Black|No Color|      Medium|    Short|        No|        No|     Healthy|  0|       1|     

### Using Triton Inference Server

Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

This notebook uses the [Python backend with a custom execution environment](https://github.com/triton-inference-server/python_backend#creating-custom-execution-environments) for Triton 24.08, using a conda-pack environment created as follows:
```
conda create -n tf-gpu -c conda-forge python=3.10.0
conda activate tf-gpu

export PYTHONNOUSERSITE=True
pip install numpy==1.26.4 tensorflow[and-cuda] conda-pack

conda-pack # tf-gpu.tar.gz
```

In [45]:
import numpy as np
import os
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import col, struct
from pyspark.sql.types import ArrayType, FloatType

In [46]:
%%bash
# copy custom model to expected layout for Triton
sudo rm -rf models
mkdir -p models
cp -r models_config/feature_columns models

# add custom execution environment
cp tf-gpu.tar.gz models

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


#### Start Triton Server on each executor

In [47]:
num_executors = 1
triton_models_dir = "{}/models".format(os.getcwd())
my_pet_classifier_dir = "{}/my_pet_classifier.keras".format(os.getcwd())
nodeRDD = sc.parallelize(list(range(num_executors)), num_executors)

def start_triton(it):
    import docker
    import time
    import tritonclient.grpc as grpcclient
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    if containers:
        print(">>>> containers: {}".format([c.short_id for c in containers]))
    else:
        container=client.containers.run(
            "nvcr.io/nvidia/tritonserver:24.08-py3", "tritonserver --model-repository=/models",
            detach=True,
            device_requests=[docker.types.DeviceRequest(device_ids=["0"], capabilities=[['gpu']])],
            name="spark-triton",
            network_mode="host",
            remove=True,
            shm_size="128M",
            volumes={
                triton_models_dir: {"bind": "/models", "mode": "ro"},
                my_pet_classifier_dir: {"bind": "/my_pet_classifier.keras", "mode": "ro"}
            }
        )
        print(">>>> starting triton: {}".format(container.short_id))

        # wait for triton to be running
        time.sleep(15)
        client = grpcclient.InferenceServerClient("localhost:8001")
        ready = False
        while not ready:
            try:
                ready = client.is_server_ready()
            except Exception as e:
                time.sleep(5)
            
    return [True]

nodeRDD.barrier().mapPartitions(start_triton).collect()

[True]

#### Run inference

In [48]:
df = spark.read.parquet("datasets/petfinder-mini")

In [49]:
df.show(5)

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized| Health|Fee|PhotoAmt|target|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
| Cat|  1|Domestic Medium Hair|Female| White|No Color|       Small|   Medium|        No|        No|Healthy|  0|       2|     1|
| Dog|  2|         Mixed Breed|Female| Black|   Brown|      Medium|   Medium|        No|        No|Healthy|  0|       3|     1|
| Dog| 18|           Dalmatian|Female| Black|   White|      Medium|   Medium|       Yes|        No|Healthy|350|       5|     1|
| Dog|  3|         Mixed Breed|Female| Black|No Color|      Medium|    Short|        No|        No|Healthy|  0|       1|     0|
| Dog|  2|         Mixed Breed|  Male| Black|   Brown|      Medium|    Short|        No|        No|Healt

In [50]:
columns = df.columns
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']


In [51]:
# remove label column
columns.remove("target")
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']


In [52]:
def triton_fn(triton_uri, model_name):
    import numpy as np
    import tritonclient.grpc as grpcclient
    
    np_types = {
      "BOOL": np.dtype(np.bool_),
      "INT8": np.dtype(np.int8),
      "INT16": np.dtype(np.int16),
      "INT32": np.dtype(np.int32),
      "INT64": np.dtype(np.int64),
      "FP16": np.dtype(np.float16),
      "FP32": np.dtype(np.float32),
      "FP64": np.dtype(np.float64),
      "FP64": np.dtype(np.double),
      "BYTES": np.dtype(object)
    }

    client = grpcclient.InferenceServerClient(triton_uri)
    model_meta = client.get_model_metadata(model_name)
    
    def predict(t, a, b, g, c1, c2, m, f, v, s, h, fee, p):
        # convert input ndarrays into a dictionary of ndarrays
        inputs = {
            "Type": t, 
            "Age": a, 
            "Breed1": b, 
            "Gender": g,
            "Color1": c1,
            "Color2": c2,
            "MaturitySize": m,
            "FurLength": f,
            "Vaccinated": v, 
            "Sterilized": s,
            "Health": h,
            "Fee": fee,
            "PhotoAmt": p
        }
        return _predict(inputs)
        
    def _predict(inputs):
        if isinstance(inputs, np.ndarray):
            # single ndarray input
            request = [grpcclient.InferInput(model_meta.inputs[0].name, inputs.shape, model_meta.inputs[0].datatype)]
            request[0].set_data_from_numpy(inputs.astype(np_types[model_meta.inputs[0].datatype]))
        else:
            # dict of multiple ndarray inputs
            request = [grpcclient.InferInput(i.name, inputs[i.name].shape, i.datatype) for i in model_meta.inputs]
            for i in request:
                i.set_data_from_numpy(inputs[i.name()].astype(np_types[i.datatype()]))
        
        response = client.infer(model_name, inputs=request)
        
        if len(model_meta.outputs) > 1:
            # return dictionary of numpy arrays
            return {o.name: response.as_numpy(o.name) for o in model_meta.outputs}
        else:
            # return single numpy array
            return response.as_numpy(model_meta.outputs[0].name)
        
        
    return predict

In [53]:
from functools import partial

# need to pass the list of columns into the model_udf
classify = predict_batch_udf(partial(triton_fn, triton_uri="localhost:8001", model_name="feature_columns"),
                             input_tensor_shapes=[[1]] * len(columns),
                             return_type=FloatType(),
                             batch_size=1024)

In [54]:
# WITHOUT custom python backend, FAILS with: Op type not registered 'DenseBincount' 
df.withColumn("preds", classify(struct(*columns))).show(truncate=10)

+----+---+----------+------+------+--------+------------+---------+----------+----------+----------+---+--------+------+----------+
|Type|Age|    Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized|    Health|Fee|PhotoAmt|target|     preds|
+----+---+----------+------+------+--------+------------+---------+----------+----------+----------+---+--------+------+----------+
| Cat|  1|Domesti...|Female| White|No Color|       Small|   Medium|        No|        No|   Healthy|  0|       2|     1| 1.9543833|
| Dog|  2|Mixed B...|Female| Black|   Brown|      Medium|   Medium|        No|        No|   Healthy|  0|       3|     1| 1.7454995|
| Dog| 18| Dalmatian|Female| Black|   White|      Medium|   Medium|       Yes|        No|   Healthy|350|       5|     1| 1.5183508|
| Dog|  3|Mixed B...|Female| Black|No Color|      Medium|    Short|        No|        No|   Healthy|  0|       1|     0|0.67013955|
| Dog|  2|Mixed B...|  Male| Black|   Brown|      Medium|    Short|        N

In [55]:
%%time
preds = df.withColumn("preds", classify(struct(*columns)))
results = preds.collect()

CPU times: user 17.2 ms, sys: 2.85 ms, total: 20 ms
Wall time: 2.5 s


In [56]:
%%time
preds = df.withColumn("preds", classify(*columns))
results = preds.collect()

CPU times: user 23.4 ms, sys: 984 μs, total: 24.4 ms
Wall time: 2.5 s


In [57]:
%%time
preds = df.withColumn("preds", classify(*[col(c) for c in columns]))
results = preds.collect()

CPU times: user 14.7 ms, sys: 4.61 ms, total: 19.3 ms
Wall time: 2.47 s


In [58]:
preds.show()

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+------------+---+--------+------+-----------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized|      Health|Fee|PhotoAmt|target|      preds|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+------------+---+--------+------+-----------+
| Cat|  1|Domestic Medium Hair|Female| White|No Color|       Small|   Medium|        No|        No|     Healthy|  0|       2|     1|  1.9543833|
| Dog|  2|         Mixed Breed|Female| Black|   Brown|      Medium|   Medium|        No|        No|     Healthy|  0|       3|     1|  1.7454995|
| Dog| 18|           Dalmatian|Female| Black|   White|      Medium|   Medium|       Yes|        No|     Healthy|350|       5|     1|  1.5183508|
| Dog|  3|         Mixed Breed|Female| Black|No Color|      Medium|    Short|        No|        No|     Healthy|  0|       1|     

#### Stop Triton Server on each executor

In [59]:
def stop_triton(it):
    import docker
    import time
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    print(">>>> stopping containers: {}".format([c.short_id for c in containers]))
    if containers:
        container=containers[0]
        container.stop(timeout=120)

    return [True]

nodeRDD.barrier().mapPartitions(stop_triton).collect()

[True]

In [60]:
spark.stop()